In [1]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"

# Load model dan konfigurasi
model_config = '/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom.cfg'
model_weights = '/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom_last.weights copy 5'
net = cv2.dnn.readNetFromDarknet(model_config, model_weights)

# Set input dan output layer untuk model
output_layers = net.getLayerNames()
output_layers = [output_layers[i - 1] for i in net.getUnconnectedOutLayers()]

# Load label dari file
with open('/Users/amariofausta/Documents/Kuliah/Semester 8/Code/Dataset/dataset/classes.names', 'r') as f:
    labels = [line.strip() for line in f.readlines()]


# Fungsi untuk melakukan deteksi objek pada setiap frame
def detect_objects(frame):
    # Konversi frame ke format blob
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Jalankan inference pada frame
    net.setInput(blob)
    layer_outputs = net.forward(output_layers)

    # Proses output layer dari model
    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # NMS untuk mengurangi overlap deteksi objek
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)

    # Gambar kotak deteksi objek pada frame
    colors = np.random.uniform(0, 255, size=(len(class_ids), 3))

    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        label = f'{labels[class_ids[i]]}: {confidences[i]:.2f}'
        color = colors[i]

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

# Fungsi untuk melakukan deteksi objek pada video
def process_video(input_video, output_video):
    clip = VideoFileClip(input_video)
    processed_clip = clip.fl_image(detect_objects)
    processed_clip.write_videofile(output_video,codec='libx264', audio=False)
    clip.reader.close()
    clip.close()

# Jalankan deteksi objek pada video
process_video('/Users/amariofausta/Documents/Kuliah/Semester 8/Video mentah/VID_20230306_110231_HSR_24010634253 (1).mp4', '/Users/amariofausta/Documents/Kuliah/Semester 8/Hasil/Hasil7.mp4')

Moviepy - Building video /Users/amariofausta/Documents/Kuliah/Semester 8/Hasil/Hasil7.mp4.
Moviepy - Writing video /Users/amariofausta/Documents/Kuliah/Semester 8/Hasil/Hasil7.mp4



Moviepy - Done !
Moviepy - video ready /Users/amariofausta/Documents/Kuliah/Semester 8/Hasil/Hasil7.mp4
